In [12]:
#import re
import sys, codecs, optparse, glob, os
import string, yaml
import random, warnings
from ToolsMoodleXML2 import *
#from cellbell import ding

def ding():
    os.system('afplay /System/Library/Sounds/Submarine.aiff')

In [13]:
nKalaba="K1"
annee="24"
dossier="/sDrive/Cours/Bordeaux/L1-LinguistiqueGenerale/00-ProjetKalaba/%s-%s"%(annee,nKalaba)
enonces=["Forme-Radical",
         "Forme-Traduction","Traduction-Radical",
#         "Forme-Traduction-MC","Traduction-Radical-MC",
#         "Forme-Proprietes",
         "Glose-Decoupe","Decoupe-Glose-MC","Decoupe-Glose"
        ]
enonces=[
         "Forme-Traduction","Traduction-Radical",
         "Proprietes-Forme",
         "Decoupe-Glose-MC","Decoupe-Glose"
        ]

# enonces=["Glose-Decoupe-MC","Proprietes-Forme-MC"]

#enonces=["Decoupe-Glose-MC"]
theme="EX-A1"

# enonces=["Theme","Version"]
# theme="Traductions"

enonces=["Proprietes-Forme"]
theme="EX-B2"

# enonces=["Ordre"]
# theme="Syntagmes"

In [14]:
from os.path import expanduser
home = expanduser("~")
repertoire=home+dossier
base=repertoire+"/"

In [15]:
def donneesEX():
    titre=[]
    result=""
    i=0
    for ligne in donneesLignes:
        i+=1
        if ligne.startswith("#EX;"):
            if result!="":
                donnees.append({"-".join(titre):result})
            titre=[]
            result={"boucle":[],"conclusion":[]}
            temp=ligne.strip().split(";")
            if len(temp)>1 and temp[1]!="":
                result["conclusion"]=temp[1:]
        elif ligne.startswith("#"):
            pass
        else:
            temp=ligne.strip().split(";")
            result["boucle"].append(temp)
            titre.append(makeChamps(configuration[u"Titre"],temp).encode('utf8'))
    donnees.append({"-".join(titre):result})

def donneesNoEX():
    if titreNoEX:
        debut=1
    else:
        debut=0
    for ligne in donneesLignes[debut:]:
        if not (ligne.strip().strip(";")=="" or ligne.startswith("#")):
            temp=ligne.strip().split(";")
            result={"boucle":[temp],"conclusion":[]}
            titre=makeChamps(configuration[u"Titre"],temp).encode('utf8')
            donnees.append({titre:result})
    return

In [16]:
for enonce in enonces:
    yamlFichier=base+theme+"-"+enonce+".yaml"
    csvFichier=base+theme+".txt"
    boolEX=False
    titreNoEX=False
            
    with open(yamlFichier, 'r') as stream:
        configuration=yaml.safe_load(stream)

    with open(csvFichier, 'r') as file:
        donneesLignes=file.readlines()

    donnees=[]
    if boolEX or donneesLignes[0].startswith("#EX;"):
    #    print "avec #EX;"
        donneesEX()
    else:
    #    print "sans #EX;"
        donneesNoEX()

        
    categorie=configuration[u"Catégorie"]
    #donnees=configuration[u"Données"]
    consigne=configuration[u"Boucle"][u"contenu"]
    boucleWrap=[configuration[u"Boucle"][u"début"],configuration[u"Boucle"]["fin"]]
    boucleCLOZE=configuration[u"Boucle"][u"Cloze"]
    globalWrap=[configuration[u"Décor"][u"fond"]+configuration[u"Décor"][u"entete"]+configuration[u"Global"][u"début"],configuration[u"Global"]["fin"]]
    if "Conclusion" in configuration:
        conclusion=[configuration[u"Conclusion"][u"début"],configuration[u"Conclusion"]["contenu"],configuration[u"Conclusion"]["fin"]]
        conclusionCLOZE=configuration[u"Conclusion"][u"Cloze"]
    else:
        conclusion=["","",""]
        conclusionCLOZE=[]
    questions=XMLSeries(categorie)
    questionsConsigne=ClozeConsigne(consigne,boucleWrap=boucleWrap,globalWrap=globalWrap,conclusion=conclusion)
    serie=ClozeSerie(configuration[u"Boucle"][u"Cloze"],conclusionCLOZE)

    for exercices in donnees:
    #    print "exercices",exercices
        for exercice in exercices:
    #        print "exercice",exercice
            exerciceBoucle=exercices[exercice]["boucle"]
            exerciceConclusion=exercices[exercice]["conclusion"]
            serie.addExercice(Exercice(exerciceBoucle,exerciceConclusion,exercice.decode('utf8')))

    for exercice in serie.makeSerie(questionsConsigne):
#        print exercice.corps
#        print
        questions.addExercice(exercice)

    xmlOutput=MoodleXML()
    xmlOutput.addQuiz(questions.getSeries())

    xmlFichier=yamlFichier.split(".")[0]+".xml"

    with codecs.open(xmlFichier, 'w',encoding="utf8") as output:
    #    print [questions.getClozes()]
        output.write(xmlOutput.getXML())

In [17]:
#xmlOutput.getXML()

In [18]:
ding()